In [73]:
import pandas as pd 
import numpy as np

excel_file = pd.ExcelFile("./data/ltcm_exhibits_data.xlsx")

sheet_names = excel_file.sheet_names
sheet_names

['Copyright', 'Exhibit 2', 'Exhibit 4']

In [74]:
ltcm = pd.read_excel('./data/ltcm_exhibits_data.xlsx', sheet_name='Exhibit 2', skiprows=2, index_col=0, parse_dates=[0]).iloc[:-4, :]
ltcm.index.name = 'Date'


ltcm = ltcm[['Net Monthly Performanceb', 'Gross Monthly Performancea']].dropna()
ltcm = ltcm.rename(columns={'Net Monthly Performanceb': 'LTCM Net', 'Gross Monthly Performancea': 'LTCM Gross'})
ltcm = ltcm.reset_index()
ltcm['Date'] = pd.to_datetime(ltcm['Date'])+ pd.tseries.offsets.MonthEnd(0)
# ltcm =  ltcm.set_index('Date')
ltcm.head()


,Date,LTCM Net,LTCM Gross
0,1994-03-31,-0.013,-0.011
1,1994-04-30,0.008,0.014
2,1994-05-31,0.053,0.068
3,1994-06-30,-0.029,-0.039
4,1994-07-31,0.084,0.116


In [83]:
rf = pd.read_excel("./data/gmo_analysis_data.xlsx", sheet_name="risk-free rate")
rf.rename(columns = {'date':'Date'},inplace=True)
rf['TBill 3M'] /= 12
rf['Date'] = pd.to_datetime(rf['Date'])
# rf = rf.set_index('Date')
rf.head()

,Date,TBill 3M
0,1989-12-31,0.006367
1,1990-01-31,0.006450
2,1990-02-28,0.006583
3,1990-03-31,0.006475
4,1990-04-30,0.006450


In [84]:
data = pd.read_excel("./data/gmo_analysis_data.xlsx", sheet_name="total returns")
data.rename(columns = {'date':'Date'},inplace=True)
# data = data.set_index('Date')
data.head()

,Date,SPY,GMWAX,GMGEX
0,1993-02-28,0.010667,NaN,NaN
1,1993-03-31,0.022408,NaN,NaN
2,1993-04-30,-0.025590,NaN,NaN
3,1993-05-31,0.026970,NaN,NaN
4,1993-06-30,0.003667,NaN,NaN


In [85]:
df = pd.merge(ltcm, rf)
df = pd.merge(df, data)
df.head()

,Date,LTCM Net,LTCM Gross,TBill 3M,SPY,GMWAX,GMGEX
0,1994-03-31,-0.013,-0.011,0.003067,-0.041900,NaN,NaN
1,1994-04-30,0.008,0.014,0.003450,0.011212,NaN,NaN
2,1994-05-31,0.053,0.068,0.003450,0.015940,NaN,NaN
3,1994-06-30,-0.029,-0.039,0.003608,-0.022880,NaN,NaN
4,1994-07-31,0.084,0.116,0.003733,0.032326,NaN,NaN


In [86]:
df = df[['Date', 'LTCM Net', 'LTCM Gross', 'TBill 3M', 'SPY']]
df = df.set_index('Date')
df.head()

,LTCM Net,LTCM Gross,TBill 3M,SPY
Date,,,,
1994-03-31,-0.013,-0.011,0.003067,-0.041900
1994-04-30,0.008,0.014,0.003450,0.011212
1994-05-31,0.053,0.068,0.003450,0.015940
1994-06-30,-0.029,-0.039,0.003608,-0.022880
1994-07-31,0.084,0.116,0.003733,0.032326


In [87]:
annualization_factor = 12
def performance_summary(return_data, annualization_factor):
    if isinstance(return_data, pd.Series):
        return_data = return_data.to_frame(name='Return')
    
    summary_stats = pd.DataFrame()

    # Compute metrics
    summary_stats['Mean'] = return_data.mean() * annualization_factor
    summary_stats['Volatility'] = return_data.std() * np.sqrt(annualization_factor)
    summary_stats['Sharpe Ratio'] = summary_stats['Mean'] / summary_stats['Volatility']

    summary_stats['Skewness'] = return_data.skew()
    summary_stats['Excess Kurtosis'] = return_data.kurtosis()
    summary_stats['VaR (0.05)'] = return_data.quantile(0.05)
    summary_stats['CVaR (0.05)'] = return_data[return_data.le(return_data.quantile(0.05))].mean()
    summary_stats['Min'] = return_data.min()
    summary_stats['Max'] = return_data.max()

    # Calculate drawdowns
    wealth_index = 1000 * (1 + return_data).cumprod()
    previous_peaks = wealth_index.cummax()
    drawdowns = (wealth_index - previous_peaks) / previous_peaks

    summary_stats['Max Drawdown'] = drawdowns.min()

    # Handling peak, bottom, and recovery for Series or DataFrame
    if isinstance(return_data, pd.DataFrame):
        summary_stats['Peak'] = [
            previous_peaks[col][:drawdowns[col].idxmin()].idxmax() for col in previous_peaks.columns
        ]
        summary_stats['Bottom'] = drawdowns.idxmin()
        
        recovery_date = []
        for col in wealth_index.columns:
            prev_max = previous_peaks[col][:drawdowns[col].idxmin()].max()
            recovery_wealth = wealth_index[col][drawdowns[col].idxmin():]
            recovery_date.append(recovery_wealth[recovery_wealth >= prev_max].index.min())
        summary_stats['Recovery'] = recovery_date
    else:
        col = 'Return'
        summary_stats['Peak'] = previous_peaks[:drawdowns.idxmin()].idxmax()
        summary_stats['Bottom'] = drawdowns.idxmin()
        
        prev_max = previous_peaks[:drawdowns.idxmin()].max()
        recovery_wealth = wealth_index[drawdowns.idxmin():]
        summary_stats['Recovery'] = recovery_wealth[recovery_wealth >= prev_max].index.min()

    return summary_stats


In [88]:
df.loc[:, ['LTCM Net', 'LTCM Gross', 'SPY']] = df.loc[:, ['LTCM Net', 'LTCM Gross', 'SPY']].subtract(df['TBill 3M'], axis=0)


In [89]:
performance_summary(df[['LTCM Net', 'LTCM Gross', 'SPY']], annualization_factor)


,Mean,Volatility,Sharpe Ratio,Skewness,Excess Kurtosis,VaR (0.05),CVaR (0.05),Min,Max,Max Drawdown,Peak,Bottom,Recovery
LTCM Net,0.192996,0.108234,1.783142,-0.879601,3.888872,-0.017566,-0.051983,-0.105133,0.080267,-0.108787,1998-04-30,1998-07-31,NaT
LTCM Gross,0.287974,0.133308,2.160224,-0.313006,1.998281,-0.019766,-0.056650,-0.105133,0.112267,-0.105133,1998-04-30,1998-06-30,NaT
SPY,0.175565,0.106577,1.647307,-0.436963,-0.198906,-0.044823,-0.048045,-0.050021,0.074980,-0.050241,1994-08-31,1994-11-30,1995-02-28


Comparing the Net Monthly Performance of LTCM and Excess returns of SPY, LTCM displays a higher return, with a very similar volatility to SPY. Thus the sharpe ratio of LTCM net of fee and other charges is slightly higher than SPY's.

The excess net returns of LTCM outperform SPY with similar volatility levels and thus have a slightly higher sharpe ratio.

However, looking at other moments, LTCM Net returns are more negatively skewed and have a significantly fatter tail compared to SPY, indicating the presence of heavy negative monthly returns over the sample period. Although, since the VaR of LTCM is lower compared to SPY, the indication is that these negative returns are fewer in frequency.


In [95]:
import statsmodels.api as sm
x = sm.add_constant(df['SPY']) 
y = df['LTCM Gross']
model = sm.OLS(y, x).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             LTCM Gross   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                 -0.012
Method:                 Least Squares   F-statistic:                    0.4499
Date:                Sun, 01 Dec 2024   Prob (F-statistic):              0.506
Time:                        15:14:43   Log-Likelihood:                 87.154
No. Observations:                  47   AIC:                            -170.3
Df Residuals:                      45   BIC:                            -166.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0222      0.006      3.539      0.001       0.010       0.035
SPY            0.1244      0.186      0.671      0.506      -0.249       0.498
==============================================================================
Omnibus:                        6.722   Durbin-Watson:                   2.335
Prob(Omnibus):                  0.035   Jarque-Bera (JB):                8.440
Skew:                          -0.364   Prob(JB):                       0.0147
Kurtosis:                       4.944   Cond. No.                         32.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [98]:
model.params['const'] * 12, model.params['SPY']

(0.26612572109515886, 0.12444852641501317)

Alpha = 0.26612572109515886, B = 0.1244, R**2 = 0.0096


In [99]:
x = sm.add_constant(df['SPY']) 
y = df['LTCM Net']
model = sm.OLS(y, x).fit()
model.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               LTCM Net   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                 -0.016
Method:                 Least Squares   F-statistic:                    0.2854
Date:                Sun, 01 Dec 2024   Prob (F-statistic):              0.596
Time:                        15:15:40   Log-Likelihood:                 96.862
No. Observations:                  47   AIC:                            -189.7
Df Residuals:                      45   BIC:                            -186.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0149      0.005      2.924      0.005       0.005       0.025
SPY            0.0806      0.151      0.534      0.596      -0.223       0.385
==============================================================================
Omnibus:                       16.812   Durbin-Watson:                   2.221
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               33.675
Skew:                          -0.916   Prob(JB):                     4.87e-08
Kurtosis:                       6.720   Cond. No.                         32.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [100]:
model.params['const'] * 12, model.params['SPY']

(0.17884208375742655, 0.08061799819039266)

- Not a closet indexer since low beta and low R**2.This is likely because they are trading fixed income primarily, and most of their trades are pairs/arbitrage trades, meaning that by definition they don't have exposure to the market.
- Very high net alpha at near 18%.

In [103]:
df['SPY_square'] = df['SPY'] ** 2 
x = sm.add_constant(df[['SPY', 'SPY_square']]) 
y = df['LTCM Net']
model = sm.OLS(y, x).fit()
# model.summary()

In [104]:
model.params['const']*12, model.params

(0.2087318792971643,
 const         0.017394
 SPY           0.125942
 SPY_square   -2.765492
 dtype: float64)

b. The quadratic factor imprroves the $R^2$ marginally, this is likely because we're just adding more features to the regression, so  must increase. So it doesn't help much.

The huge negative beta on SPY squared is a feature of the factor. The monthly returns are small, thus the squared returns are even smaller, thus the beta has to be larger in magnitude to fit these small returns properly.

c. 
Since the beta to SPY squared returns is negative, LTCM's market exposure behaves as if it were short the market options. The beta to SPY can be interpreted as the delta of the market option and the beta to SPY squared as the gamma to market options. Since the gamma of an option is always positive, LTCM seems to be short the positive gamma or short the market options.

BUT: this is not significant, so we can't really say anything about it.

d.

For a big monthly return, the negative beta for SPY Squared would lead to heavy underperformance of LTCM returns. Big market movements would lead to big underperformance of LTCM. Thus, LTCM seems to be taking on a negative exposure to market volatility underperforming big deviations in the market and the performance not being impacted much by small deviations.

In [108]:
df['SPY_Put'] = np.maximum(-0.03 - df['SPY'], 0)
df['SPY_Call'] = np.maximum(df['SPY'] - 0.03, 0)

x = sm.add_constant(df[['SPY', 'SPY_Put', 'SPY_Call']]) 
y = df['LTCM Net']
model = sm.OLS(y, x).fit()
model.params['const']*12, model.params

(0.16121955831741897,
 const       0.013435
 SPY         0.339976
 SPY_Put     0.937522
 SPY_Call   -0.716244
 dtype: float64)

In [109]:
y = df['LTCM Gross']
model = sm.OLS(y, x).fit()
model.params['const']*12, model.params

(0.2374550147140381,
 const       0.019788
 SPY         0.488780
 SPY_Put     1.407126
 SPY_Call   -0.968403
 dtype: float64)

b. Long puts short calls based on the betas 

c. We are more long puts than we are short calls, based on the magnitude of the betas. So the put-like behavior dominates.

d.

We know that they are short volatility, but we cannot conclude this based on this regression alone, and, in reality, this regression actually implies the opposite. Namely, being long options is being long vol. So, we are long a lot more put options than call options, meaning that in a vacuum, we are long vol. However, this depends on the strikes of the options. So if the call-like factor is closer to ATM than the put-like factor, then we might actually be short vol, since the calls will have a higher vega than the puts. However, if the strikes are symmetric (equally OTM), then we are long vol.

But; we can say that we are long downside vol, and short upside vol.

# FX Carry Trade

In [12]:
excel_file = pd.ExcelFile("./data/fx_rf_data.xlsx")

sheet_names = excel_file.sheet_names
sheet_names

['risk-free rates', 'exchange rates']

In [13]:
risk_free_rates = pd.read_excel('./data/fx_rf_data.xlsx', sheet_name='risk-free rates')
risk_free_rates.index = risk_free_rates['date']
risk_free_rates = risk_free_rates.drop(['date'],axis=1)
for col in risk_free_rates.columns:
    risk_free_rates[col] = risk_free_rates[col]
    risk_free_rates['log_'+col] = np.log(1+risk_free_rates[col])
risk_free_rates

,USD,JPY,EUR,GBP,MXN,CHF,log_USD,log_JPY,log_EUR,log_GBP,log_MXN,log_CHF
date,,,,,,,,,,,,
2008-01-01,0.044000,0.00478,0.042500,0.057000,0.075,0.020645,0.043059,0.004769,0.041622,0.055435,0.072321,0.020435
2008-01-02,0.041563,0.00478,0.036000,0.054750,0.075,0.020645,0.040722,0.004769,0.035367,0.053304,0.072321,0.020435
2008-01-03,0.043463,0.00478,0.039400,0.056000,0.075,0.021500,0.042545,0.004769,0.038644,0.054488,0.072321,0.021272
2008-01-04,0.042188,0.00478,0.040650,0.055000,0.075,0.020950,0.041322,0.004769,0.039846,0.053541,0.072321,0.020734
2008-01-07,0.042188,0.00493,0.041000,0.054600,0.075,0.020200,0.041322,0.004918,0.040182,0.053162,0.072321,0.019999
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-25,0.048700,0.00227,0.032000,0.049575,0.105,0.008450,0.047551,0.002267,0.031499,0.048385,0.099845,0.008414
2024-10-28,0.048650,0.00228,0.032088,0.050500,0.105,0.008400,0.047504,0.002277,0.031584,0.049266,0.099845,0.008365
2024-10-29,0.048625,0.00228,0.031750,0.050500,0.105,0.008350,0.047480,0.002277,0.031256,0.049266,0.099845,0.008315


In [14]:
fx_rates = pd.read_excel('./data/fx_rf_data.xlsx', sheet_name='exchange rates')
fx_rates.index = fx_rates['date']
fx_rates = fx_rates.drop(['date'],axis=1)
for col in fx_rates.columns:
    fx_rates['log_'+col] = np.log(fx_rates[col])
fx_rates

,JPY,EUR,GBP,MXN,CHF,log_JPY,log_EUR,log_GBP,log_MXN,log_CHF
date,,,,,,,,,,
2008-01-01,0.008957,1.4592,1.9864,0.09183,0.8824,-4.715320,0.377888,0.686324,-2.387816,-0.125110
2008-01-02,0.009119,1.4715,1.9808,0.09162,0.8933,-4.697395,0.386282,0.683501,-2.390106,-0.112833
2008-01-03,0.009148,1.4750,1.9710,0.09187,0.9002,-4.694220,0.388658,0.678541,-2.387381,-0.105138
2008-01-04,0.009209,1.4743,1.9740,0.09154,0.9022,-4.687574,0.388183,0.680062,-2.390979,-0.102919
2008-01-07,0.009159,1.4696,1.9704,0.09176,0.8957,-4.693018,0.384990,0.678237,-2.388579,-0.110150
...,...,...,...,...,...,...,...,...,...,...
2024-10-25,0.006565,1.0796,1.2962,0.05003,1.1536,-5.026003,0.076591,0.259437,-2.995132,0.142887
2024-10-28,0.006524,1.0812,1.2972,0.04989,1.1555,-5.032268,0.078072,0.260208,-2.997935,0.144533
2024-10-29,0.006521,1.0819,1.3015,0.04984,1.1530,-5.032728,0.078719,0.263517,-2.998937,0.142367


In [15]:
fx_spot_list = ['log_GBP'
                ,'log_EUR'
                ,'log_CHF'
                ,'log_JPY']

fx_hldg_lst = []
for fx in fx_spot_list:
    print(fx)
    fx_hldg_excess_ret = fx_rates[fx] - fx_rates[fx].shift(1) + risk_free_rates[fx].shift(1) - risk_free_rates['log_USD'].shift(1)
    fx_hldg_summary = performance_summary(fx_hldg_excess_ret.to_frame().dropna(), 365)
    fx_hldg_lst.append(fx_hldg_summary)

fx_hldg_perf_summary = pd.concat(fx_hldg_lst)
perf = fx_hldg_perf_summary.T
perf.columns = fx_spot_list 
perf

log_GBP
log_EUR
log_CHF
log_JPY


,log_GBP,log_EUR,log_CHF,log_JPY
Mean,-0.063196,-2.476384,-4.688252,-4.274324
Volatility,0.196042,0.238755,0.278514,0.325795
Sharpe Ratio,-0.322358,-10.372081,-16.833071,-13.119678
Skewness,0.433109,0.289161,0.195637,-1.241317
Excess Kurtosis,3.116699,0.473687,8.486205,0.911931
VaR (0.05),-0.016861,-0.027229,-0.038338,-0.051971
CVaR (0.05),-0.020924,-0.030475,-0.042269,-0.055873
Min,-0.083643,-0.041149,-0.094564,-0.069421
Max,0.061093,0.053725,0.192459,0.033561
Max Drawdown,-0.999996,-1.0,-1.0,-1.0


- The currencies all recorded negative returns on average relative to USD with signficiant volatility experienced for CHF and JPY at the highest. This coincided with the extremely negative sharpe ratios for EUR, JPY and CHF in increasing order.
- JPY is heavily negatively skewed, while CHF stands out for its extreme kurtosis indicating fat tails and risks of extreme outcomes. 

- UIP states that the mean return of these currencies positions should be zero as the change in the spot fx rate is completely explained by the changes in risk free rates. However, none of the mean returns for the currencies are 0.
- Over the sample, a long position in CHF would have offered positive returns
- All currencies earned negative excess returns in USD terms

In [16]:
fx_hldg_reg = {'Alpha': [], 'Beta': [], 'R-Squared': []}

for fx in fx_spot_list:
    # Compute factor and strategy
    factor = (risk_free_rates['log_USD'] - risk_free_rates[fx]).shift(1)
    strat = fx_rates[fx].diff().dropna()
    
    # Align factor and strat after dropping NaN values
    factor = factor.loc[strat.index].dropna()  # Ensure indices match
    
    # Add constant for intercept
    factor = sm.add_constant(factor)

    # Run regression
    model = sm.OLS(strat, factor).fit()
    
    # Append regression results to dictionary
    fx_hldg_reg['Alpha'].append(model.params['const'])  # Intercept (Alpha)
    fx_hldg_reg['Beta'].append(model.params[factor.columns[1]])  # Beta (Slope)
    fx_hldg_reg['R-Squared'].append(model.rsquared)  # R-Squared

# Convert results to a DataFrame
fx_hldg_reg_summary = pd.DataFrame(fx_hldg_reg, index=fx_spot_list)
fx_hldg_reg_summary = fx_hldg_reg_summary.T
fx_hldg_reg_summary

,log_GBP,log_EUR,log_CHF,log_JPY
Alpha,-0.000100,-0.000124,0.000016,-0.000087
Beta,0.027484,0.008577,0.003575,0.001452
R-Squared,0.001517,0.000288,0.000047,0.000014


- Beta > 0: as foreign risk free rate increase, USD strengthens. 
- nil
- Strongest for GBP based on the R square

In [20]:
fx_prem_lst = []
for fx in fx_spot_list:
    fx_er_usd = (risk_free_rates['log_USD'].shift(1) - risk_free_rates[fx].shift(1)).to_frame('ER_over_USD')
    expected_fx_premium = float(fx_hldg_reg_summary.loc['Alpha',fx]/annualization_factor) + (fx_er_usd.loc[:,['ER_over_USD']]  * float(fx_hldg_reg_summary.loc['Beta',fx] - 1))
    expected_fx_premium = expected_fx_premium.rename(columns={'ER_over_USD':fx})
    positive_premium =  len(expected_fx_premium[expected_fx_premium[fx] > 0])
    fx_prem_lst.append(pd.DataFrame([[positive_premium,len(expected_fx_premium),positive_premium*100/len(expected_fx_premium)]],columns=['Months - Positive Premium','Total Months','Frequency(%)-Positive Premium'],index=[fx]))
fx_premium = pd.concat(fx_prem_lst)
fx_premium

,Months - Positive Premium,Total Months,Frequency(%)-Positive Premium
log_GBP,2342,4393,53.312087
log_EUR,1154,4393,26.269064
log_CHF,115,4393,2.617801
log_JPY,8,4393,0.182108


- GBP (most consistently have a positive FX risk premium). JPY (most consistently have a negative FX risk premium)
- Since JPY returns are negative, an improvement in the carry trade would be to short the JPY and invest in the USD risk free rate. With the forecast based on the beta being positive, this means that as JPY risk free rate increases, we could foreast the USD strengthening and thus yield a positive risk premia. 